In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ConnectButton（）{
console.log（" Connect push"）; 
document.querySelector（"＃top-toolbar> colab-connect- 
按钮"）.shadowRoot.querySelector（"＃connect"）。click（）
} 
setInterval（ConnectButton，60000）; 

In [ ]:
import os
os.listdir('/content/drive/MyDrive/bert_cnn_blstm_crf')

In [ ]:
!pip install kashgari

In [ ]:
from google.colab import files
uploaded=files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

In [ ]:
def read_corpus(corpus_path):
    """
    read corpus and return the list of samples
    :param corpus_path:
    :return: data
    """
    pad_size=10
    windows_size=75
    data = []
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()
    sent_, tag_ = [], []
    for line in lines:
        [char, label] = line.rstrip().split(' ')
        sent_.append(char)
        tag_.append(label)
    for cur_idx in range(pad_size,len(sent_)-pad_size,windows_size):
        chars=sent_[cur_idx-pad_size:cur_idx+windows_size+pad_size]
        tags=tag_[cur_idx-pad_size:cur_idx+windows_size+pad_size]
        data.append([chars,tags])

    return data
def get_data(data):
    x_data=[]
    y_data=[]
    for x,y in data:
        x_data.append(x)
        y_data.append(y)
    return x_data,y_data

In [ ]:
open('/content/drive/MyDrive/bert_cnn_blstm_crf/model_info.json')

In [ ]:
test_data = read_corpus('test')
test_x,test_y=get_data(test_data)
path='/content/drive/MyDrive/bert_cnn_blstm_crf/model_info.json'
import kashgari
# train_data.extend(test_data)
# np.random.seed(200)
# np.random.shuffle(train_data)
# train=train_data[:round(len(train_data)*args.ratio1)]
# dev=train_data[round(len(train_data)*args.ratio1):round(len(train_data)*args.ratio2)]
# test=train_data[round(len(train_data)*args.ratio2):]
loaded_model=kashgari.utils.load_model(path)
    # _,label_predict=loaded_model.evaluate(test_x,test_y)   
loaded_model.predict(test_x[0])


# 卷积神经网络函数实现


In [ ]:
from mxnet import autograd,nd
from mxnet.gluon import nn

# def corr2d(X,K):
#   h,w=K.shape
#   Y=nd.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
#   for i in range(Y.shape[0]):
#     for j in range(Y.shape[1]):
#       Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
#   return Y

from mxnet import autograd, nd
from mxnet.gluon import nn

def corr2d(X, K):  # 本函数已保存在d2lzh包中方便以后使用
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y


In [ ]:
X = nd.ones((6, 8))
X[2:4, :] = 0
X

In [ ]:

K = nd.array([[-1],[1]])
h, w = K.shape
corr2d(X,K)

In [ ]:
X=nd.array([[0,1,2],[3,4,5],[6,7,8]])
K=nd.array([[0,1],[2,3]])
corr2d(X,K)



In [ ]:
class Conv2D(nn.Block):
  def __init__(self,kernel_size,**kwargs):
    super(Conv2D,self).__init__(**kwargs)
    self.weight=self.params.get('weight',shape=kernel_size)
    self.bias=self.params.get('bias',shape=(1,))
  def forward(self,x):
    return corr2d(x,self.weight.data())+self.bias.data()
    


In [ ]:
X=nd.ones((6,8))
X[:,2:6]=0
X

In [ ]:
K=nd.array([[1,-1]])
K

In [ ]:
Y=corr2d(X,K)

In [ ]:
corr2d()

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
from google.colab import files
uploaded=files.upload()
for fn in uploaded.keys():
  print('{name},{length}'.format(name=fn,length=len(uploaded[fn])))

In [ ]:
from google.colab import files
uploaded=files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

In [ ]:
open('test.csv')

In [ ]:
!pip install d2lzh

In [ ]:
open('train_augmented.txt')

In [ ]:
!pip install mxnet

In [ ]:
import d2lzh as d2l
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import data as gdata,loss as gloss,nn
import numpy as np
import pandas as pd


In [ ]:
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [ ]:
train_data.shape

In [ ]:
train_data.iloc[0:4,[0,1,2,3,-3,-2,-1]]

In [ ]:
all_features=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))

In [ ]:
numeric_features=all_features.dtypes[all_features.dtypes!='object'].index
all_features[numeric_features]=all_features[numeric_features].apply(lambda x:(x-x.mean())/(x.std()))
all_features[numeric_features]=all_features[numeric_features].fillna(0)

In [ ]:
all_features=pd.get_dummies(all_features,dummy_na=True)
all_features.shape

In [ ]:
n_train=train_data.shape[0]
train_features=nd.array(all_features[:n_train].values)
test_features=nd.array(all_features[n_train:].values)
train_labels=nd.array(train_data.SalePrice.values).reshape((-1,1))


In [ ]:
loss=gloss.L2Loss()
def get_net():
  net=nn.Sequential()
  net.add(nn.Dense(1))
  net.initialize()
  return net

In [ ]:
def log_rmse(net,features,labels):
  clipped_preds=nd.clip(net(features),1,float('inf'))
  rmse=nd.sqrt(2*loss(clipped_preds.log(),labels.log()).mean())
  return rmse.asscalar()

In [ ]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = gdata.DataLoader(gdata.ArrayDataset(
        train_features, train_labels), batch_size, shuffle=True)
    # 这里使用了Adam优化算法
    trainer = gluon.Trainer(net.collect_params(), 'adam', {
        'learning_rate': learning_rate, 'wd': weight_decay})
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

In [ ]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = nd.concat(X_train, X_part, dim=0)
            y_train = nd.concat(y_train, y_part, dim=0)
    return X_train, y_train, X_valid, y_valid

In [ ]:
def k_fold(k, X_train, y_train, num_epochs,
           learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        train_l_sum += train_ls[-1]
        valid_l_sum += valid_ls[-1]
        if i == 0:
            d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse',
                         range(1, num_epochs + 1), valid_ls,
                         ['train', 'valid'])
        print('fold %d, train rmse %f, valid rmse %f'
              % (i, train_ls[-1], valid_ls[-1]))
    return train_l_sum / k, valid_l_sum / k

In [ ]:

k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print('%d-fold validation: avg train rmse %f, avg valid rmse %f'
      % (k, train_l, valid_l))

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
open('rel_dict.json')

In [ ]:
import json

In [ ]:
def read_txt_file(file_path):
    data=[]
    dic=load()
    with open(file_path, 'r', encoding='utf-8') as f:
        content=[]
        for line in f.readlines():
            if line is not '\n':
                line=line.rstrip()
                line=line.replace(' ', '')
                content.append(line)
    # labels, texts,word= [], [],[]
    word=[]
    for line in content:
        parts = line.split('\t')
        label, text = parts[0], ''.join(parts[1:])
        label=dic[int(label)]
        for i in text:
            word.append(i)
        # labels.append(label)
        # texts.append(word)
        data.append((label,word))
        word=[]
        
    return data
def load():
  with open('rel_dict.json','r',encoding='utf-8') as h:
    data=json.load(h)
    data={v:k for k,v in data.items()}
  return data


In [ ]:
def get_data(data):
  x_data=[]
  y_data=[]
  for idx in data:
    x_data.append(idx[1])
    y_data.append(idx[0])
  return x_data,y_data

In [ ]:
data=read_txt_file('train.txt')

import numpy as np
np.random.seed(200)
np.random.shuffle(data)
train_data=data[:round(len(data)*0.8)]
val_data=data[round(len(data)*0.6):round(len(data)*0.8)]
test_data=data[round(len(data)*0.8):]


In [ ]:
len(data)

In [ ]:
train_x,train_y=get_data(train_data)
val_x,val_y=get_data(val_data)
test_x,test_y=get_data(test_data)


In [ ]:
!pip install kashgari

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
import kashgari
from tensorflow.python import keras
from kashgari.tasks.classification import CNN_Model

from kashgari.callbacks import EvalCallBack
from kashgari.embeddings import BareEmbedding,BertEmbedding



In [ ]:
from typing import Dict,Any
from tensorflow import keras

from kashgari.tasks.classification.abc_model import ABCClassificationModel
from kashgari.layers import L

import logging
logging.basicConfig(level='DEBUG')


class CNN_BiLSTM_Model(ABCClassificationModel):
    """Bidirectional LSTM Sequence Labeling Model"""

    @classmethod
    def default_hyper_parameters(cls) -> Dict[str, Dict[str, Any]]:
        """
        Get hyper parameters of model
        Returns:
            hyper parameters dict
        """
        return {
            'layer_blstm1': {
                'units': 128,
                'return_sequences': True
            },
            'layer_blstm2': {
                'units': 128,
                'return_sequences': False
            },
            'layer_dropout': {
                'rate': 0.4
            },
            'layer_time_distributed': {},
            'layer_output': {

            }
        }

    def build_model_arc(self):
        """
        build model architectural
        """
        output_dim = len(self.processor.label2idx)
        config = self.hyper_parameters
        embed_model = self.embedding.embed_model

        # Define your layers
        layer_blstm1 = L.Bidirectional(L.LSTM(**config['layer_blstm1']),
                                       name='layer_blstm1')
        layer_blstm2 = L.Bidirectional(L.LSTM(**config['layer_blstm2']),
                                       name='layer_blstm2')

        layer_dropout = L.Dropout(**config['layer_dropout'],
                                  name='layer_dropout')

        layer_time_distributed = L.Dense(output_dim, **config['layer_output'])

        # You need to use this actiovation layer as final activation
        # to suppor multi-label classification
        layer_activation = self._activation_layer()
        tensor=embed_model.output
        kernel_sizes=[7,7,7]
        for kernel_size in kernel_sizes:
            tensor=L.Conv1D(128,kernel_size=kernel_size)(tensor)
            tensor=L.MaxPool1D()(tensor)
        lstm=layer_blstm1(embed_model.output)
        tensor=L.Conv1D(128,kernel_size=7)(tensor)
        tensor=L.GlobalMaxPool1D()(tensor)  
    
        output=L.concatenate([tensor,lstm],axis=1)
        output_tensor=L.Dense(output_dim,activation='softmax')(output)
  
        
        # Define tensor flow
        
        # Init model
        self.tf_model = keras.Model(embed_model.inputs, output_tensor)


In [ ]:
import tensorflow as tf
tf.keras.optimizers.Adam()

In [ ]:
model=CNN_Model()
model.build_model(train_x,train_y)

optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile_model(optimizer=optimizer)
tf_board_callback=keras.callbacks.TensorBoard()
eval_callback=EvalCallBack(model,val_x,val_y,step=1)
history=model.fit(train_x,train_y,val_x,val_y,batch_size=8,epochs=40,callbacks=[eval_callback,tf_board_callback])
model.evaluate(test_x,test_y)

model.evaluate(test_x,test_y)

# 新段落

In [ ]:
len(data)

In [ ]:
import matplotlib.pyplot as plt
plt.plot()
epochs = len(history.history['loss'])
plt.plot(range(epochs), history.history['loss'], label='loss')
plt.plot(range(epochs), history.history['val_loss'], label='val_loss')
plt.legend()
plt.savefig("loss_acc.svg")


In [ ]:
from google.colab import files
files.download('loss_acc.svg')

In [ ]:
import sklearn

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
!wget -O /usr/share/fonts/truetype/liberation/simhei.ttf "https://www.wfonts.com/download/data/2014/06/01/simhei/chinese.simhei.ttf"

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
zhfont = mpl.font_manager.FontProperties(fname='/usr/share/fonts/truetype/liberation/simhei.ttf')
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
zhfont = mpl.font_manager.FontProperties(fname='/usr/share/fonts/truetype/liberation/simhei.ttf')
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.xlabel(u'词语', fontproperties=zhfont)
plt.ylabel(u'特征重要度', fontproperties=zhfont)
plt.xticks(fontproperties=zhfont)
plt.bar(imp_plot['name'],imp_plot['imp'])

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

def plot_confusion_matrix(cm, savename, title='Confusion Matrix'):

    plt.figure(figsize=(12, 8), dpi=100)
    np.set_printoptions(precision=2)

    # 在混淆矩阵中每格的概率值
    ind_array = np.arange(len(classes))
    x, y = np.meshgrid(ind_array, ind_array)
    for x_val, y_val in zip(x.flatten(), y.flatten()):
        c = cm[y_val][x_val]
        if c > 0.001:
            plt.text(x_val, y_val, "%0.0f" % (c,), color='red', fontsize=15, va='center', ha='center')
    
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.viridis)
    plt.title(title)
    plt.colorbar()
    xlocations = np.array(range(len(classes)))
    plt.xticks(xlocations, classes, rotation=90)
    plt.yticks(xlocations, classes)
    plt.ylabel('Actual label')
    plt.xlabel('Predict label')
     
    #指定默认字体  
    #解决负号'-'显示为方块的问题   

    # offset the tick
    tick_marks = np.array(range(len(classes))) + 0.5
    plt.gca().set_xticks(tick_marks, minor=True,fontproperties=zhfont)
    plt.gca().set_yticks(tick_marks, minor=True,fontproperties=zhfont)
    plt.gca().xaxis.set_ticks_position('none')
    plt.gca().yaxis.set_ticks_position('none')
    plt.grid(True, which='minor', linestyle='-')
    plt.gcf().subplots_adjust(bottom=0.15)
    
    # show confusion matrix
    plt.savefig(savename, format='svg')
    plt.show()

In [ ]:
with open('rel_dict.json') as f:
  data=json.load(f)
rel=[]

In [ ]:
len(data.keys())

In [ ]:
# classes表示不同类别的名称，比如这有6个类别
classes = data.keys()
line=open('rel_dict.json')
  # 6个类别，随机生成50个样本
y_pred= model.predict(test_x)  # 样本实际标签
  # 将前10个样本的值进行随机更改
 # 样本预测标签
y_true=test_y
# 获取混淆矩阵
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, 'confusion_matrix.svg', title='confusion matrix')

In [ ]:
from google.colab import files
files.download('confusion_matrix.svg')

In [ ]:
from sklearn.metrics import confusion_matrix    # 生成混淆矩阵函数
import matplotlib.pyplot as plt    # 绘图库
import numpy as np
import tensorflow as tf

In [ ]:
def plot_confusion_matrix(cm, labels_name, title):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]    # 归一化
    plt.imshow(cm, interpolation='nearest')    # 在特定的窗口上显示图像
    plt.title(title)    # 图像标题
    ind_array = np.arange(len(labels_name))
    x, y = np.meshgrid(ind_array, ind_array)
    for x_val, y_val in zip(x.flatten(), y.flatten()):
        c = cm[y_val][x_val]
        if c > 0.001:
            plt.text(x_val, y_val, "%0.0f" % (c,), color='red', fontsize=15, va='center', ha='center')
    plt.colorbar()
    num_local = np.array(range(len(labels_name)))    
    plt.xticks(num_local, labels_name, rotation=90,fontproperties=zhfont)    # 将标签印在x轴坐标上
    plt.yticks(num_local, labels_name,fontproperties=zhfont)    # 将标签印在y轴坐标上
    plt.ylabel('True label')    
    plt.xlabel('Predicted label')

In [ ]:
!wget -O /usr/share/fonts/truetype/liberation/simhei.ttf "https://www.wfonts.com/download/data/2014/06/01/simhei/chinese.simhei.ttf"

In [ ]:
y_true=test_y
classes = data.keys()
y_pred=model.predict(test_x)
cm = confusion_matrix(y_true, y_pred)
print(cm)
# [[100   1   0   1   6   0   0]
#  [  2 111   3   0   2   1  24]
#  [  0   2  68   5   4   3   2]
#  [  2   0   1 120   7  26   0]
#  [  2   5   3   2 120  11  14]
#  [  2   0   2  12   8 115   1]
#  [  2  25   0   1  14   4 302]]

In [ ]:
plot_confusion_matrix(cm, classes, "HAR Confusion Matrix")
plt.savefig('/HAR_cm.svg', format='svg')
plt.show()

In [ ]:
from google.colab import files
files.download('/HAR_cm.svg')

In [ ]:
!pip install keras_radam

In [ ]:
from kashgari.tasks.classification import BiLSTM_Model
from tensorflow import keras


model1=BiLSTM_Model()
model1.build_model(train_x,train_y)
optimizer=keras.optimizers.Adam(learning_rate=0.001)
model1.compile_model(optimizer=optimizer)
tf_board_callback=keras.callbacks.TensorBoard()
eval_callback=EvalCallBack(model,val_x,val_y,step=1)
history=model1.fit(train_x,train_y,val_x,val_y,batch_size=8,epochs=40,callbacks=[eval_callback,tf_board_callback])
model1.evaluate(test_x,test_y)

model.evaluate(test_x,test_y)

In [ ]:
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm,target_names,title='Confusion matrix',cmap=plt.cm.Reds,#设置混淆矩阵的颜色主题normalize=True):
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(15, 12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
   
    plt.show()

def plot_confuse(model, x_val, y_val, labels):
    predictions = model.predict_classes(x_val,batch_size=batch)
    truelabel = y_val.argmax(axis=-1)   # 将one-hot转化为label
    conf_mat = confusion_matrix(y_true=truelabel, y_pred=predictions)
    plt.figure()
    plot_confusion_matrix(conf_mat, normalize=False,target_names=labels,title='Confusion Matrix')



In [ ]:
!pip install kashgari